# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv()
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/08/06/outsmart/',
 'https://edwarddonner.com/2024/06/26/choosing-the-right-llm-resources/

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/11/13/llm-engineering-resources/
https://edwarddonner.com/2024/10/16/from-software-engineer-to-ai-data-scientist-resources/
https://edwarddonner

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
anthropic = Website("https://anthropic.com")
anthropic.links

['/',
 '/claude',
 '/team',
 '/enterprise',
 '/api',
 '/pricing',
 '/research',
 '/company',
 '/careers',
 '/news',
 'https://www.anthropic.com/research#entry:8@1:url',
 'https://www.anthropic.com/claude',
 'https://claude.ai/',
 '/api',
 '/news/3-5-models-and-computer-use',
 '/claude/sonnet',
 '/claude/haiku',
 '/news/claude-for-enterprise',
 '/research/constitutional-ai-harmlessness-from-ai-feedback',
 '/news/core-views-on-ai-safety',
 '/jobs',
 '/',
 '/claude',
 '/api',
 '/team',
 '/pricing',
 '/research',
 '/company',
 '/customers',
 '/news',
 '/careers',
 'mailto:press@anthropic.com',
 'https://support.anthropic.com/',
 'https://status.anthropic.com/',
 '/supported-countries',
 'https://twitter.com/AnthropicAI',
 'https://www.linkedin.com/company/anthropicresearch',
 'https://www.youtube.com/@anthropic-ai',
 '/legal/consumer-terms',
 '/legal/commercial-terms',
 '/legal/privacy',
 '/legal/aup',
 '/responsible-disclosure-policy',
 'https://trust.anthropic.com/']

In [11]:
get_links("https://anthropic.com")

{'links': [{'type': 'home page', 'url': 'https://anthropic.com/'},
  {'type': 'about page', 'url': 'https://anthropic.com/company'},
  {'type': 'team page', 'url': 'https://anthropic.com/team'},
  {'type': 'careers page', 'url': 'https://anthropic.com/careers'},
  {'type': 'research page', 'url': 'https://anthropic.com/research'},
  {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'},
  {'type': 'api page', 'url': 'https://anthropic.com/api'},
  {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [12]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [13]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'company page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Claude
Overview
Team
Enterprise
API
Pricing
Research
Company
Careers
News
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.5 Sonnet
Claude 3.5 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Start using Claude to drive efficiency and create new revenue streams.
Learn more
Announcements
Introducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku
Oct 22, 2024
Model updates
3.5 Sonnet
3.5 Haiku
Our Work
Product
Claude for Enterprise
Sep 4, 2024
Alignment
·
Research
Constitutional AI: Harmlessness fr

In [14]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [15]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:20_000] # Truncate if more than 20,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}]}


"You are looking at a company called: Anthropic\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nClaude\nOverview\nTeam\nEnterprise\nAPI\nPricing\nResearch\nCompany\nCareers\nNews\nAI\nresearch\nand\nproducts\nthat put safety at the frontier\nClaude.ai\nMeet Claude 3.5 Sonnet\nClaude 3.5 Sonnet, our most intelligent AI model, is now available.\nTalk to Claude\nAPI\nBuild with Claude\nStart using Claude to drive efficiency and create new revenue streams.\nLearn more\nAnnouncements\nIntroducing computer use, a new Claude 3.5 Sonnet, and Claude 3.5 Haiku\nOct 22, 2024\nModel updates\n3.5 Sonnet\n3.5 Haiku\nOur Work\nProduct\nClaude for Enterprise\nSep 4, 2024\nAlignment\n·\nResearch\nConstitutional AI: Harmlessness from AI Feedback\nDec 15, 2022\nAnnouncements\nCore Views on AI Safety: When, Why, What, and How\nMar 8, 2023\nWo

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("ESRamada", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}, {'type': 'enterprise page', 'url': 'https://anthropic.com/enterprise'}, {'type': 'api page', 'url': 'https://anthropic.com/api'}, {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}, {'type': 'news page', 'url': 'https://anthropic.com/news'}]}


# Anthropic Company Brochure

## Overview
**Anthropic** is an AI safety and research company based in San Francisco, committed to developing reliable, interpretable, and steerable AI systems. We aim to make AI systems that people can trust and can positively impact society.

## Our Mission
At Anthropic, we strive to ensure that transformative AI helps individuals and societies flourish. We build frontier AI systems and regularly share insights to foster a collaborative approach to AI safety across various domains.

## Our Core Values
1. **Here for the Mission**: We prioritize our mission to build safe and beneficial AI systems that can tackle emerging challenges.
2. **Unusually High Trust**: Communication is rooted in a foundation of honesty, emotional maturity, and intellectual openness.
3. **One Big Team**: Collaboration transcends departmental boundaries with everyone working toward a shared goal.
4. **Do the Simple Thing That Works**: We emphasize practical solutions and empirical evidence in our approach.

## Products & Services
### Claude AI
- **Claude** is our flagship AI assistant designed to improve productivity by leveraging team knowledge, making work more efficient and creative.
- Claude is available via an API, helping enterprises drive efficiency and create new revenue streams.

### Enterprise Solutions
- Anthropic provides tailored AI solutions to empower organizations, enabling effective collaboration and utilization of internal knowledge while ensuring data security.

## Research & Development
Our interdisciplinary research teams focus on cutting-edge AI methodologies, exploring safety techniques from interpretability to societal impact analysis. We prioritize sharing our findings with the broader AI community and stakeholders to promote industry-wide safety.

## Company Culture
Anthropic fosters a culture of collaboration, innovation, and support. Our team comprises diverse professionals from various fields, including machine learning, policy, and operations. We value each member's contributions and believe in a flat structure where every voice is heard. Our focus on mental and physical health includes comprehensive benefits, flexible time off, and support for employee well-being.

## Career Opportunities
We are always looking for dedicated individuals to join our mission to build safe AI systems. We welcome diverse backgrounds and experiences, and we do not require prior machine learning experience for many roles. Our hiring process is designed to be inclusive and fair.

### Benefits of Working at Anthropic
- **Health & Wellness**: Comprehensive insurance options, inclusive family benefits, and generous parental leave.
- **Compensation & Support**: Competitive salaries and equity packages, along with flexible wellness stipends and professional development incentives.
- **Collaborative Environment**: Opportunities for cross-team collaboration on diverse projects.

**Join us in shaping the future of safe AI! [See open roles on our Careers Page](#)**

## Our Customers
Anthropic collaborates with businesses, nonprofits, and civil society organizations to ensure that our AI solutions meet their needs effectively. Our clients share positive feedback on the productivity boosts achieved through using Claude.

## Connect with Us
Stay up-to-date with our latest advancements and announcements through our social media channels:
- [Twitter](#)
- [LinkedIn](#)
- [YouTube](#)

For more information, visit our website at [Anthropic](https://www.anthropic.com).

---

*Anthropic PBC - Building safer and more reliable AI.*

In [19]:
create_brochure("ESRamada", "https://www.esramada.pt/")

Found links: {'links': [{'type': 'about page', 'url': 'http://www.esramada.pt/index.php/escola/historia/breve-historia'}, {'type': 'company page', 'url': 'http://www.esramada.pt/index.php/escola/historia/moinho-das-covas'}, {'type': 'careers page', 'url': 'http://www.esramada.pt/index.php/recrutamento'}, {'type': 'location page', 'url': 'http://www.esramada.pt/index.php/localizacao'}, {'type': 'services page', 'url': 'http://www.esramada.pt/index.php/escola/servicos'}, {'type': 'partners page', 'url': 'http://www.esramada.pt/index.php/escola/parceiros'}]}


# Brochure: Escola Secundária da Ramada (ESRamada)

## About Us
Founded in 1980, the Escola Secundária da Ramada is a prestigious secondary school located in Loures, Portugal. With a rich history and commitment to education, we serve the community with a robust curriculum aimed at nurturing students from the 9th to 12th grades.

### Key Milestones:
- **1980**: Construction of the school begins.
- **1980**: Official opening of the school to the local community.
- **1996**: Restoration of the Moinho das Covas, a historic windmill, serving as both a cultural landmark and educational resource.

We pride ourselves on being a school that adapts to modern educational needs while respecting our cultural heritage.

## Our Culture
### Community-Oriented
At ESRamada, our mission is to foster a supportive and collaborative environment for all students. We actively promote mentorship programs where experienced students guide newcomers, creating a welcoming and attentive community. Initiatives like our food and hygiene product collection campaigns highlight our commitment to social responsibility and helping those in need.

### Innovation in Education
We are committed to integrating technology and innovative practices in our teaching methodology. Programs like **Inovar+** empower both teachers and students to utilize modern tools for effective learning experiences.

## Our Customers
ESRamada caters to a diverse body of students, parents, and educators. Our community includes:
- **Students**: We focus on holistic development, equipping young individuals with essential skills for future endeavors.
- **Parents**: Working closely with families to ensure their children receive the best education and support.
- **Educational Partners**: Collaborating with local government and organizations, such as the Câmara Municipal de Loures, to foster educational projects like the restoration of cultural sites.

## Careers at ESRamada
We are always on the lookout for passionate individuals dedicated to education and youth development. Opportunities include:
- Teaching positions across various subjects.
- Administrative and support roles.
- Volunteer mentorship and program development.

Our dynamic team is committed to continuous improvement and innovation in education, making ESRamada a vibrant place to work.

### Join Us!
If you're interested in shaping young lives and contributing to a thriving educational community, explore career opportunities on our [Recruitment Page](#).

---

For more information about our school, educational programs, or opportunities to get involved, please visit our [website](#) or contact us directly. We look forward to welcoming you to the Escola Secundária da Ramada community!

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [20]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [21]:
stream_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://anthropic.com/company'}, {'type': 'careers page', 'url': 'https://anthropic.com/careers'}, {'type': 'team page', 'url': 'https://anthropic.com/team'}, {'type': 'api page', 'url': 'https://anthropic.com/api'}, {'type': 'pricing page', 'url': 'https://anthropic.com/pricing'}, {'type': 'research page', 'url': 'https://anthropic.com/research'}]}


# Anthropic: Advancing Safe AI

Welcome to **Anthropic**, a pioneering public benefit corporation committed to making artificial intelligence safe, reliable, and beneficial for humanity. Based in San Francisco, we are at the forefront of AI research and product development, focusing on systems that prioritize interpretability and steerability.

---

## Our Mission

At Anthropic, we believe that artificial intelligence can transform the world for the better. Our purpose is clear: to build AI systems that people can trust and rely on for a safer and more productive future. We engage in cutting-edge research to evaluate the risks and opportunities associated with AI technologies.

## Products

### Meet Claude
Introducing **Claude** — Anthropic's suite of AI models, including the latest** Claude 3.5 Sonnet** and **Claude 3.5 Haiku**. These advanced systems help businesses streamline workflows, generate insights, and boost productivity while ensuring safety and reliability.

### Build with Claude
Our API allows developers to harness the power of Claude models for user-facing applications, ensuring everyone can leverage state-of-the-art AI capabilities in their work.

---

## Company Culture

### Values
Our work culture is defined by collaboration, mutual trust, and a commitment to our mission:

- **Here for the Mission**: We focus on AI advancements that benefit society.
- **Unusually High Trust**: We foster an environment built on goodwill, honesty, and emotional maturity.
- **One Big Team**: Everyone’s input is valued, emphasizing collaboration across disciplines.
- **Do the Simple Thing That Works**: We prioritize practical solutions and empirical methods over complexity.

### Team
Our diverse team comprises researchers, engineers, policy experts, and operations specialists, all collaborating to push the boundaries of AI advancements. We treat safety as a scientific discipline, continuously refining our approach through research and innovation.

---

## Customers
Our products and research have augmented various sectors, including businesses, nonprofits, and civil society groups. Customers benefit from tools that enhance productivity — from marketing to engineering — allowing teams to achieve results faster and more effectively.

---

## Careers at Anthropic

**Join Us!** We are always on the lookout for talented individuals who are passionate about shaping the future of AI. 

### What We Offer
- **Health & Wellness**: Comprehensive health, dental, and vision plans, along with generous mental health support.
- **Family Benefits**: Inclusive parental leave and fertility support.
- **Flexible Work**: Opportunities for remote work and flexible schedules, as well as stipends for home office support.
- **Culture of Growth**: An environment that values unique perspectives, offering robust mentorship opportunities and professional development.

### Hiring Process
We embrace a thorough yet fair hiring approach, focusing on diverse backgrounds and experiences. Our interview process emphasizes collaboration and understanding. Whether you come from a technical, business, or policy background, we want to hear from you!

---

## Get in Touch
For more information about our products, research, or careers, please visit our website or connect with us through our social media channels.

Join us in building a safer, more reliable future with AI!

---
### **Connect with Us**
- **[Twitter](#)**
- **[LinkedIn](#)**
- **[YouTube](#)**

### **Explore More**
- [Claude](#)
- [API](#)
- [Careers](#)
- [Research](#)

---

© 2024 Anthropic PBC. All Rights Reserved.

In [22]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}]}



# Hugging Face Brochure

## About Us
**Hugging Face** is the AI community building the future of machine learning (ML). Our mission is to democratize access to ML technologies, making it easier for developers, researchers, and businesses to collaborate on models, datasets, and applications.

## What We Offer
- **Models**: Over 400,000 state-of-the-art ML models available for various tasks and domains.
- **Datasets**: Access to more than 100,000 datasets to facilitate research and development.
- **Spaces**: A collaborative platform where users can run and share their ML applications and demos with the world.
- **Enterprise Solutions**: Tailored plans for organizations needing enhanced security, dedicated support, and advanced analytics.

## Community & Culture
At Hugging Face, we believe in collaboration and transparency. Our vibrant community includes over 50,000 organizations, from notable companies like Google, Microsoft, and Amazon to innovative startups. We foster a culture of open source contributions and continuous learning, ensuring everyone can express their ideas, share their work, and grow together.

### Featured Companies Using Hugging Face
- **Meta**
- **Intel**
- **Grammarly**
- **Toyota Research Institute**

## Why Choose Us?
- **Open Source Being**: Build on top of a foundation that is continuously enhanced by the community.
- **Scalability**: Our infrastructure supports projects of all sizes, from startups to enterprise-level applications.
- **Innovative Tools**: Explore the latest advancements in ML including GPT architecture and pandemic-proof models.
  
## Careers at Hugging Face
Join our talented team of over 225 members and be part of the AI revolution. We offer a range of career opportunities across various domains. Our work environment promotes creativity, autonomy, and flexibility, allowing you to thrive professionally and personally.

### Current Openings
- Machine Learning Engineers
- Data Scientists
- Product Managers
- Software Developers

To explore job openings, [click here](https://huggingface.co/jobs).

## Join Us!
Become part of the Hugging Face community today and accelerate your machine learning journey. Whether you are an individual eager to learn or an organization looking to harness the power of AI, we have the tools and resources you need.

### Contact Us
For inquiries or to get started, visit our [website](https://huggingface.co) or connect with us on [Twitter](https://twitter.com/huggingface).

---

Together, let's shape the future of AI!


<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 2 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!
            </span>
        </td>
    </tr>
</table>